In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Conv2D, Dense
from tensorflow.keras import layers
import keras.backend as K
import sys
from tensorflow.keras.datasets import cifar10

np.set_printoptions(threshold=sys.maxsize)
gpus = tf.config.experimental.list_physical_devices('XLA_GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'XLA_GPU')

# Block

In [2]:
def Conv_bn(inputs, filters, kernel_size, strides = 1,padding = 'same', activation = 'relu'):
    
    x = layers.Conv2D(filters, kernel_size, kernel_initializer = 'he_normal', padding = 'same', strides = strides)(inputs)
    x = layers.BatchNormalization()(x)
    if activation:
        x = layers.Activation('relu')(x)
    
    return x

In [3]:
def SE_block(inputs, reduction_ratio=16):
    ch_input = K.int_shape(inputs)[-1]
    ch_reduced = ch_input//reduction_ratio
    
    # Squeeze
    x = GlobalAveragePooling2D()(inputs) # Eqn.2
    
    # Excitation
    x = Dense(ch_reduced, kernel_initializer='he_normal', activation='relu', use_bias=False)(x) # Eqn.3
    x = Dense(ch_input, kernel_initializer='he_normal', activation='sigmoid', use_bias=False)(x) # Eqn.3
    
    x = layers.Reshape((1, 1, ch_input))(x)
    x = layers.Multiply()([inputs, x]) # Eqn.4
    
    return x

In [4]:
def SE_residual_block(inputs, filters, strides = 1, reduction_ratio = 16):
    
    filter1, filter2, filter3 = filters
    
    # conv block -> conv + bn + ac / conv + bn + ac / conv + bn
    x = Conv_bn(inputs, filter1, (1, 1), strides = strides)
    x = Conv_bn(x, filter2, (3, 3))
    x = Conv_bn(x, filter3, (1, 1), activation = None)
    
    x = SE_block(x, reduction_ratio)
    
    p_input = Conv_bn(inputs, filter3, (1, 1), strides = strides, activation=None)
    x = layers.add([p_input, x])
    x = layers.Activation(activation='relu')(x)
    
    return x

In [5]:
def stage_block(inputs, filter_sizes, blocks, reduction_ratio = 16, stage = ''):
    strides = 2 if stage != '2' else 1
    
    x = SE_residual_block(inputs, filter_sizes, strides, reduction_ratio)
    
    for i in range(blocks - 1):
        x = SE_residual_block(x, filter_sizes, reduction_ratio=reduction_ratio)
        
    return x

In [6]:
def SE_ResNet101(inputs, classes = 10):
    stage_1 = Conv_bn(inputs, 64, (7, 7), strides = 2, padding = 'same')
    stage_1 = layers.MaxPooling2D((3, 3), strides = 2, padding = 'same')(stage_1)
    
    stage_2 = stage_block(stage_1, [64, 64, 256], 3, reduction_ratio=16, stage='2')
    stage_3 = stage_block(stage_2, [128, 128, 512], 4, reduction_ratio=16, stage='3') # (28, 28, 512)
    stage_4 = stage_block(stage_3, [256, 256, 1024], 23, reduction_ratio=16, stage='4') # (14, 14, 1024)
    stage_5 = stage_block(stage_4, [512, 512, 2048], 3, reduction_ratio=16, stage='5') # (7, 7, 2048)
    
    gap = GlobalAveragePooling2D()(stage_5)
#     gap = layers.Flatten()(stage_5)
    
    outputs = Dense(classes, activation = 'softmax')(gap)
        
    model = keras.Model(inputs = inputs, outputs = outputs, name = 'SE-ResNet101')
    
    return model

In [7]:
def Up_scaling(data, shape):
    x = tf.image.resize(data, shape)
    
    return x

# Model

In [8]:
input_shape = keras.Input(shape = (224, 224, 3))

In [9]:
model = SE_ResNet101(input_shape, classes = 10)

In [10]:
model.summary()

Model: "SE-ResNet101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [12]:
X_train = x_train / 255.0
X_test = x_test / 255.0

X_valid, y_valid = X_train[10000:12500], y_train[10000:12500]

X_train = X_train[:10000]
y_train = y_train[:10000]

In [13]:
def Upscaling_data(data, shape):
    
    new_array = tf.image.resize(data, (224, 224))
        
    new_array = np.array(new_array)
    
    return new_array

In [14]:
X_train = Upscaling_data(X_train, input_shape)
X_test = Upscaling_data(X_test, input_shape)
X_valid = Upscaling_data(X_valid, input_shape)

In [15]:
print('X_train_shape : ', X_train.shape, 'X_test_shape : ', X_test.shape, 'X_valid_shape : ', X_valid.shape)

X_train_shape :  (10000, 224, 224, 3) X_test_shape :  (10000, 224, 224, 3) X_valid_shape :  (2500, 224, 224, 3)


In [16]:
Adam = tf.keras.optimizers.Adam(learning_rate=0.001, decay = 0.00001)

model.compile(optimizer = 'adam', 
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics = ['accuracy'])

In [17]:
model.fit(X_train, y_train, epochs=30, validation_data = (X_valid, y_valid), batch_size = 8)

Epoch 1/30
1250/1250 [==============================] - 357s 285ms/step - loss: 2.2409 - accuracy: 0.1587 - val_loss: 11.3064 - val_accuracy: 0.1084
Epoch 2/30
1250/1250 [==============================] - 356s 285ms/step - loss: 2.2889 - accuracy: 0.1354 - val_loss: 2.2343 - val_accuracy: 0.1456
Epoch 3/30
1250/1250 [==============================] - 357s 286ms/step - loss: 2.2435 - accuracy: 0.1535 - val_loss: 2.1709 - val_accuracy: 0.1708
Epoch 4/30
1250/1250 [==============================] - 358s 286ms/step - loss: 2.2155 - accuracy: 0.1682 - val_loss: 2.1584 - val_accuracy: 0.2100
Epoch 5/30
1250/1250 [==============================] - 358s 287ms/step - loss: 2.1232 - accuracy: 0.1967 - val_loss: 2.1939 - val_accuracy: 0.1772
Epoch 6/30
1250/1250 [==============================] - 360s 288ms/step - loss: 2.2233 - accuracy: 0.1607 - val_loss: 2.3777 - val_accuracy: 0.1164
Epoch 7/30
1250/1250 [==============================] - 358s 286ms/step - loss: 2.2520 - accuracy: 0.1480 - val

In [18]:
history = model.evaluate(X_test, y_test)

313/313 [==============================] - 75s 241ms/step - loss: 1.2964 - accuracy: 0.5502


In [19]:
model.save('./model.h5')

In [20]:
new_model = tf.keras.models.load_model('./model.h5')

In [33]:
bb = model.predict(X_test)

In [34]:
nn = new_model.predict(X_test)

In [35]:
mm = np.argmax(bb, axis = 1)

In [36]:
mm.shape

(10000,)

In [38]:
mm[0:5]

array([3, 8, 8, 0, 6], dtype=int64)

In [39]:
y_test[0:5]

array([[3],
       [8],
       [8],
       [0],
       [6]], dtype=uint8)

In [40]:
pp = np.argmax(nn, axis = 1)

In [41]:
pp.shape

(10000,)

In [42]:
pp[0:5]

array([3, 8, 8, 0, 6], dtype=int64)